Install the necessary library

In [ ]:
!pip install accelerate -U datasets transformers[torch] evaluate

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Load dataset from huggingface and train,test,val split

In [ ]:
from datasets import load_dataset

dataset = load_dataset("....", token=True)
wbc = dataset

In [ ]:
train_test_split = wbc['train'].train_test_split(test_size=0.3)
wbc['train'] = train_test_split['train']
wbc['test'] = train_test_split['test']

validation_test_split = wbc['test'].train_test_split(test_size=0.5)
wbc['val'] = validation_test_split['train']
wbc['test'] = validation_test_split['test']


Assign label to id

In [ ]:
label_names = ['Basophil','Eosinophil','Lymphocyte','Monocyte','Neutrophil']
id2label = {id: label for id, label in enumerate(label_names)}
label2id = {label: id for id, label in id2label.items()}

Load pre-trained Vision transformer

In [ ]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

Normalization the data

In [ ]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [ ]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

In [ ]:
wbc = wbc.with_transform(transforms)

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

Set the model and arguments for fine-tuning

In [ ]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels= 5,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

In [ ]:
training_args = TrainingArguments(
    output_dir="....",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=64,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=64,
    num_train_epochs=16,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

Define custom trainer class for optimization training with weight loss function

In [ ]:
import torch
from torch import nn
from transformers import Trainer

class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        # Initialize the weights
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights).to(self.model.device)
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        if self.class_weights is not None:
            loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
        else:
            loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss



Set weight and train the model

In [ ]:
class_weights = [36.09, 10.19, 3.01, 13.66, 1.00]

trainer = CustomTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=wbc["train"],
    eval_dataset=wbc["val"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    class_weights=class_weights,  # Pass the class weights
)
trainer.train()

Evaluate the model with test set

In [ ]:
test_results = trainer.evaluate(wbc["test"])
print(test_results)